In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [2]:
## load data
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [3]:
# function to read image
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (256,256))
    return img

In [4]:
## set path for images
TRAIN_PATH = 'Data/train_img/'
TEST_PATH = 'Data/test_img/'

In [5]:
# load data
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|█████████████████████████████████████| 1732/1732 [00:06<00:00, 268.01it/s]


In [6]:
# normalize images
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [7]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [8]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [9]:
y_train = to_categorical(y_train)

In [10]:
#Transfer learning with Inception V3 
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
print('Model loaded.')

Model loaded.


In [11]:
# Number of product classifications
print(y_train.shape[1])

26


In [13]:
## set model architechture 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

 
#removed to check
model = Model(inputs=base_model.input, outputs=add_model(base_model.output))



# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.

    
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [16]:
batch_size = 16   # 32 # tune it
epochs = 2 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        #rescale=1./255,# newly added
        #shear_range=0.2,# newly added
        #zoom_range=0.2,# newly added
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [17]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/2
199/200 [============================>.] - ETA: 185s - loss: 3.2354 - acc: 0.0000e+0 - ETA: 164s - loss: 3.0436 - acc: 0.0938    - ETA: 157s - loss: 3.0696 - acc: 0.083 - ETA: 152s - loss: 3.0296 - acc: 0.093 - ETA: 150s - loss: 3.0218 - acc: 0.087 - ETA: 148s - loss: 3.0439 - acc: 0.072 - ETA: 146s - loss: 3.0392 - acc: 0.080 - ETA: 145s - loss: 3.0262 - acc: 0.085 - ETA: 143s - loss: 3.0515 - acc: 0.083 - ETA: 142s - loss: 3.0710 - acc: 0.081 - ETA: 141s - loss: 3.0692 - acc: 0.085 - ETA: 140s - loss: 3.0512 - acc: 0.099 - ETA: 139s - loss: 3.0435 - acc: 0.096 - ETA: 138s - loss: 3.0419 - acc: 0.098 - ETA: 137s - loss: 3.0493 - acc: 0.100 - ETA: 136s - loss: 3.0570 - acc: 0.093 - ETA: 135s - loss: 3.0719 - acc: 0.091 - ETA: 134s - loss: 3.0763 - acc: 0.093 - ETA: 134s - loss: 3.0783 - acc: 0.098 - ETA: 133s - loss: 3.0805 - acc: 0.109 - ETA: 132s - loss: 3.0767 - acc: 0.113 - ETA: 131s - loss: 3.0820 - acc: 0.110 - ETA: 130s - loss: 3.0805 - acc: 0.111 - ETA: 130s - loss: 3

C:\Users\skumarravindran\AppData\Local\Continuum\Anaconda2\envs\py35gpu1\lib\site-packages\keras\callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


200/200 [==============================] - 146s - loss: 3.0701 - acc: 0.1216   
Epoch 2/2
200/200 [==============================] - ETA: 136s - loss: 3.2865 - acc: 0.0000e+0 - ETA: 140s - loss: 3.1555 - acc: 0.0938    - ETA: 141s - loss: 3.0952 - acc: 0.104 - ETA: 141s - loss: 3.0996 - acc: 0.125 - ETA: 140s - loss: 3.0700 - acc: 0.150 - ETA: 140s - loss: 3.0584 - acc: 0.156 - ETA: 140s - loss: 3.0639 - acc: 0.142 - ETA: 139s - loss: 3.0352 - acc: 0.140 - ETA: 139s - loss: 3.0287 - acc: 0.131 - ETA: 138s - loss: 3.0285 - acc: 0.150 - ETA: 137s - loss: 3.0208 - acc: 0.147 - ETA: 137s - loss: 3.0209 - acc: 0.145 - ETA: 136s - loss: 3.0226 - acc: 0.139 - ETA: 135s - loss: 3.0210 - acc: 0.142 - ETA: 135s - loss: 3.0215 - acc: 0.137 - ETA: 134s - loss: 3.0395 - acc: 0.136 - ETA: 133s - loss: 3.0560 - acc: 0.128 - ETA: 132s - loss: 3.0388 - acc: 0.131 - ETA: 132s - loss: 3.0356 - acc: 0.134 - ETA: 131s - loss: 3.0462 - acc: 0.128 - ETA: 130s - loss: 3.0575 - acc: 0.122 - ETA: 130s - loss: 3

In [56]:
## predict test data
predictions = model.predict(x_test)


In [58]:
# get labels
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

AxisError: axis 1 is out of bounds for array of dimension 1

In [59]:
## make submission
sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('sub_vgg1.csv', index=False) ## ~0.59